In [3]:
 !pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 16.0MB/s 
     |████████████████████████████████| 890kB 19.7MB/s 
     |████████████████████████████████| 3.0MB 24.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e23835f672561e734f606b704b0a74b4099f875e3b209e61dca62a5492275f77
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive
import pandas as pd
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
import torch

drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [5]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [6]:
import os
from transformers import BertForSequenceClassification, BertTokenizer

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/model_save/")
tokenizer = BertTokenizer.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/model_save/")

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [7]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from sklearn.metrics import matthews_corrcoef
import torch

def getScores(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Set the batch size.  
  batch_size = 32  

  # Create the DataLoader.
  prediction_data = TensorDataset(input_ids, attention_masks, labels)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  # Prediction on test set

  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    print(predictions[0])

  # Combine the results across all batches. 
  flat_predictions = np.concatenate(predictions, axis=0)

  # For each sample, pick the label (0 or 1) with the higher score.
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

  '''
  print(len(flat_predictions))

  print(flat_predictions[0])
  print(flat_predictions[1])

  '''

  # Combine the correct labels for each batch into a single list.
  flat_true_labels = np.concatenate(true_labels, axis=0)

  #print(len(flat_true_labels))

  cnt = 0
  for i in range (len(flat_predictions)):
    if flat_predictions[i] != flat_true_labels[i]:
        cnt = cnt + 1
  #print(cnt)

  #print(flat_predictions)
  #print(flat_true_labels)

  # Calculate the MCC
  mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

  print('Total MCC: %.3f' % mcc)

  return flat_predictions

def main():
  #This cell gets you to np arrays that contain all the sentences and the values coordinating to them

  #principal sentences
  df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/principal_filtered20.csv", delimiter=',', header=None, names=['number', 'sentence', 'element_name', 'element_type', 'reg_name', 'reg_type', 'reg_label'])
  training_labels = []
  for _ in range (len(df1['number'])):
      training_labels.append(1)
  df1['training_labels'] = training_labels

  values = df1.sentence.values
  labels = df1.training_labels.values

  values = np.delete(values,0)
  labels = np.delete(labels,0)


  # data principal
  df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/data_filtered20.csv", delimiter=',', header=None, names=['number', 'sentence', 'element_name', 'element_type', 'reg_name', 'reg_type', 'reg_label'])
  training_labels = []
  for _ in range (len(df2['number'])):
      training_labels.append(1)
  df2['training_labels'] = training_labels

  bad = df2.sentence.values
  zeros = df2.training_labels.values

  bad = np.delete(bad,0)
  zeros = np.delete(zeros,0)

  done = False
  added = 0
  idx = 0

  '''
  values = values.tolist()
  labels = labels.tolist()
  bad = bad.tolist()
  '''
  limit = len(values)

  while not done:
      if bad[idx] not in values:
          added =  added + 1
          values = np.append(values,bad[idx], axis=None)
          labels = np.append(labels,0, axis=None)
      if added == limit:
          done = True
      idx = idx + 1   

  i = len(values) - 1
  while i > 0:
      try:
          values[i] = values[i].replace("++++ ", "")
      except:
          None
      if str(values[i]) == 'nan':
          values = np.delete(values,i)
          labels = np.delete(labels,i)
      i = i - 1
  print(len(values))
  print(len(labels))
  print(getScores(values,labels))

if __name__ == "__main__":
  main()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Streaming output truncated to the last 5000 lines.
 [-2.4649026   2.023152  ]
 [-2.3510578   1.9039425 ]
 [-1.4576466   1.2521662 ]
 [-2.3812327   1.938522  ]
 [-2.4903462   1.9923955 ]]
[[-1.4822488   1.3331763 ]
 [-1.3910146   1.3107423 ]
 [-2.633478    2.1380012 ]
 [ 0.08450446 -0.20324194]
 [-2.3546987   1.9364429 ]
 [ 1.6070671  -1.2606145 ]
 [-2.402723    1.94269   ]
 [-2.34319     1.8867338 ]
 [-2.4093766   1.941434  ]
 [-2.4093766   1.941434  ]
 [ 0.6206048  -0.62265354]
 [-2.4604342   1.952644  ]
 [-0.77442706  0.702588  ]
 [-2.376123    1.8937056 ]
 [-2.3300693   1.9457096 ]
 [-2.5512745   2.0479078 ]
 [ 1.4850396  -1.1611282 ]
 [-2.423836    1.9261222 ]
 [-2.423836    1.9261222 ]
 [-1.7093904   1.6154747 ]
 [-2.5144446   2.0021367 ]
 [-2.641819    2.20312   ]
 [-2.393171    1.9415227 ]
 [-2.420204    1.8959736 ]
 [-2.444394    1.9633545 ]
 [ 1.0660446  -0.9373803 ]
 [-1.8448799   1.6705577 ]
 [-2.4649026   2.023152  ]
 [-2.3510578   1.9039425 ]
 [-1.4576466   1.2521662 ]
 [-